In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import Xception, InceptionV3, VGG16, ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import os # Importing the os module

# Specifying train, valid and test directory paths
# Use os.path.join to create correct paths
train_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'train')
valid_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'validation')
test_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'test')

# Rescaling the image
train_batches = ImageDataGenerator(rescale=1.0 / 255.0)
valid_batches = ImageDataGenerator(rescale=1.0 / 255.0)
test_batches = ImageDataGenerator(rescale=1.0 / 255.0)

In [ ]:
# Train data generator
train_gen = train_batches.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    classes=['angry', 'happy', 'neutral', 'sad'],
    batch_size=32
)

# Valid data generator
valid_gen = valid_batches.flow_from_directory(
    directory=valid_path,
    target_size=(224, 224),
    classes=['angry', 'happy', 'neutral', 'sad'],
    batch_size=32
)

# Test data generator
test_gen = test_batches.flow_from_directory(
    directory=test_path,
    target_size=(224, 224),
    classes=['angry', 'happy', 'neutral', 'sad'],
    batch_size=32,
    shuffle=False
)

Found 1764 images belonging to 4 classes.
Found 376 images belonging to 4 classes.
Found 380 images belonging to 4 classes.


In [ ]:
#Initializing models
from tensorflow.keras.applications import DenseNet121

model_densenet = DenseNet121(include_top=False, input_shape=(224, 224, 3))

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Freezing the layers so that they cannot be trained again
names = [model_densenet]
for model in names:
    # Iterating all the layers in the pre-trained model
    for layer in model.layers:
        # Making trainable layers set to False
        layer.trainable = False

In [ ]:
# Fine-tuning the pre-trained models
output_classes = len(train_gen.class_indices)

# Custom DenseNet121 model
custom_densenet_model = Sequential([
    model_densenet,
    GlobalAveragePooling2D(),
    Dense(output_classes, activation='softmax')
])

In [ ]:
# Compiling the models
models = [custom_densenet_model]

for model in models:
    model.compile(
        optimizer='Adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

MODIFIED LAYER

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import ReLU
from sklearn.metrics import accuracy_score
import numpy as np
import time  # Added for execution time measurement

model_densenet = DenseNet121(include_top=False, input_shape=(224, 224, 3))

# Define the DenseNet121 model with a custom top
base_model = DenseNet121(weights='imagenet', include_top=False, input_tensor=Input(shape=(224, 224, 3)))
base_model.trainable = False  # Freeze base for initial training

# >>> Custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)

x = Dense(512)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.3)(x)

x = Dense(256)(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = Dropout(0.3)(x)

output = Dense(4, activation='softmax')(x)  # Final output for 4 emotion classes

# Final model
custom_densenet_model = Model(inputs=base_model.input, outputs=output)

# Compile
custom_densenet_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# >>> Start timer
start_time = time.time()

# Train the DenseNet121 model
print(">>> Training DenseNet121 model:")

result = custom_densenet_model.fit(
    train_gen,
    validation_data=valid_gen,
    epochs=100,
    callbacks=[early_stopping],
    verbose=1
)

# Store the training history
model_histories = [(result.history, "DenseNet121")]

# Evaluate on the test data
print(">>> Evaluating DenseNet121 on the Test data:")
test_pred = custom_densenet_model.predict(test_gen)
test_labels = test_gen.classes
test_accuracy = accuracy_score(np.argmax(test_pred, axis=1), test_labels)
print(f">>> Test Accuracy for DenseNet121: {test_accuracy * 100:.2f}%.")

# >>> End timer and print execution time
end_time = time.time()
execution_time = end_time - start_time
minutes, seconds = divmod(execution_time, 60)
print(f">>> Total execution time: {int(minutes)} minutes and {int(seconds)} seconds.")

>>> Training DenseNet121 model:


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_858']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


56/56 ━━━━━━━━━━━━━━━━━━━━ 494s 8s/step - accuracy: 0.2748 - loss: 1.7363 - val_accuracy: 0.3218 - val_loss: 1.3873
Epoch 2/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 159ms/step - accuracy: 0.3864 - loss: 1.3742 - val_accuracy: 0.5239 - val_loss: 1.1911
Epoch 3/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 8s 149ms/step - accuracy: 0.4996 - loss: 1.1405 - val_accuracy: 0.6915 - val_loss: 1.0084
Epoch 4/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 161ms/step - accuracy: 0.5797 - loss: 0.9749 - val_accuracy: 0.8005 - val_loss: 0.8526
Epoch 5/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 157ms/step - accuracy: 0.6389 - loss: 0.9070 - val_accuracy: 0.8404 - val_loss: 0.7269
Epoch 6/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 10s 150ms/step - accuracy: 0.7067 - loss: 0.7842 - val_accuracy: 0.8856 - val_loss: 0.6151
Epoch 7/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 161ms/step - accuracy: 0.7860 - loss: 0.6448 - val_accuracy: 0.9229 - val_loss: 0.5261
Epoch 8/100
56/56 ━━━━━━━━━━━━━━━━━━━━ 9s 164ms/step - accuracy: 0.7958 - loss: 0.6062 - val_accuracy: 0.9335

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_858']
Received: inputs=Tensor(shape=(32, 224, 224, 3))
  warnings.warn(msg)


12/12 ━━━━━━━━━━━━━━━━━━━━ 112s 9s/step
>>> Test Accuracy for DenseNet121: 99.74%.
>>> Total execution time: 18 minutes and 52 seconds.
